### This has to be automated to be triggered only when the new data arrives in to_processed folder and after saving to delta the file must be moved to processed folder.

In [ ]:
configs = {"fs.azure.account.auth.type":"OAuth",
"fs.azure.account.oauth.provider.type":"org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id":"<client-id>",
"fs.azure.account.oauth2.client.secret":'<secret-key>',
"fs.azure.account.oauth2.client.endpoint":"https://login.microsoftonline.com/<tenant-id>/oauth2/token"}


mount_name = "/mnt/ecomproject2"
mounted_paths = [mount.mountPoint for mount in dbutils.fs.mounts()]

if mount_name not in mounted_paths:
    dbutils.fs.mount(
    source = "abfss://landing-zone-2@ecomadlsbusa.dfs.core.windows.net",     #container@storageacc
    mount_point= mount_name,
    extra_configs=configs
                )

else:
    print(f"{mount_name} is already mounted.")




/mnt/ecomproject2 is already mounted.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.appName("Ecomdatapipelinechallenge").getOrCreate()

In [ ]:
#Read parquet file
user_df = spark.read.format("parquet")\
               .option("header","true")\
               .option("inferSchema","true")\
               .load("/mnt/ecomproject2/users-raw-2/to_processed")

In [ ]:
user_df.printSchema()

root
 |-- identifierHash: string (nullable = true)
 |-- type: string (nullable = true)
 |-- country: string (nullable = true)
 |-- language: string (nullable = true)
 |-- socialNbFollowers: string (nullable = true)
 |-- socialNbFollows: string (nullable = true)
 |-- socialProductsLiked: string (nullable = true)
 |-- productsListed: string (nullable = true)
 |-- productsSold: string (nullable = true)
 |-- productsPassRate: string (nullable = true)
 |-- productsWished: string (nullable = true)
 |-- productsBought: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- civilityGenderId: string (nullable = true)
 |-- civilityTitle: string (nullable = true)
 |-- hasAnyApp: string (nullable = true)
 |-- hasAndroidApp: string (nullable = true)
 |-- hasIosApp: string (nullable = true)
 |-- hasProfilePicture: string (nullable = true)
 |-- daysSinceLastLogin: string (nullable = true)
 |-- seniority: string (nullable = true)
 |-- seniorityAsMonths: string (nullable = true)
 |-- senio

In [ ]:
user_df = spark.read.format("parquet")\
               .option("header","true")\
               .option("inferSchema","true")\
               .load("/mnt/ecomproject2/users-raw-2/to_processed")

In [ ]:
user_df.write.format("delta")\
        .mode("overwrite")\
        .option("mergeSchema", "true")\
        .save("/mnt/delta/tables/ecom2/bronze/users")

In [ ]:
#Move processed files into processed files folder
source_directory      = "/mnt/ecomproject2/users-raw-2/to_processed/"
destination_directory = "/mnt/ecomproject2/users-raw-2/processed/"

files = dbutils.fs.ls(source_directory)
for file in files:
    if file.isFile():
        # Construct source and destination paths
        source_path = file.path
        destination_path = destination_directory + source_path.split("/")[-1]
        
        # Move the file
        dbutils.fs.mv(source_path, destination_path)